In [1]:
import fs
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
INTERIM_PROJECT_DIR = fs.open_fs("../../data/interim/project")
INTERIM_FILE_NAME = INTERIM_PROJECT_DIR.getsyspath("use_to_divide.csv")

PROCESSED_PROJECT_DIR = fs.open_fs("../../data/processed/project")
TRAIN_DIR = PROCESSED_PROJECT_DIR.getsyspath("train.csv")
TEST_DIR = PROCESSED_PROJECT_DIR.getsyspath("test.csv")
VAL_DIR = PROCESSED_PROJECT_DIR.getsyspath("validation.csv")

In [3]:
df_to_divide = pd.read_csv(INTERIM_FILE_NAME)

df_to_divide.shape

(5960, 14)

In [4]:
data_without_outliers = df_to_divide[df_to_divide['outlier_flag']=='Not Outlier'].drop(['outlier_flag'], axis=1)

Split datasets

In [5]:
data_without_outliers.shape

(5947, 13)

In [6]:
categoric_features = ['BAD', 'REASON', 'JOB']
data_without_outliers[categoric_features] = data_without_outliers[categoric_features].apply(lambda col: col.astype('category'))

In [7]:
train_df, test_df = train_test_split(data_without_outliers, test_size=0.5,random_state=1)
test_df, val_df = train_test_split(test_df, test_size=0.3, random_state=1)

In [8]:
display(len(train_df), len(test_df), len(val_df)) 

2973

2081

893

In [9]:
per_data = []

# Iterate over columns of object dtype
for col in val_df.select_dtypes(include=['object', 'category']):
    # Calculate value counts as percentages
    percentages = val_df[col].value_counts(normalize=True) * 100
    # Iterate over each value and its percentage
    for category, percentage in percentages.items():
        # Append data as a dictionary
        per_data.append({'Column': col, 'Category': category, 'Percentage': percentage})

# Create a new DataFrame from the list of dictionaries
#train_df_per = pd.DataFrame(per_data)
#test_df_per = pd.DataFrame(per_data)
val_df_per = pd.DataFrame(per_data)

In [10]:
train_df.to_csv(TRAIN_DIR, index=False, encoding="utf-8")
test_df.to_csv(TEST_DIR, index=False, encoding="utf-8")
val_df.to_csv(VAL_DIR, index=False, encoding="utf-8")